In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# ML
from kernels import gaussian_kernel
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import KernelCenterer
from sklearn.model_selection import GridSearchCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer

# Utilities
import h5py
from tempfile import mkdtemp
from shutil import rmtree
from tqdm.notebook import tqdm
from tools import load_json, save_json
import project_utils as utils
from copy import deepcopy

In [3]:
# Load train and test sets
train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/train.idxs', dtype=int)
cv_idxs = np.loadtxt('../Processed_Data/DEEM_330k/cv_5.idxs', dtype=int)

In [4]:
# BEGIN TMP

In [5]:
# Train and test sets (random DEEM)
from numpy.random import default_rng
idxs_delete = np.loadtxt('../Processed_Data/DEEM_330k/10kJmol_error.idxs', dtype=int)
deem_10k_idxs = np.loadtxt('../Processed_Data/DEEM_330k/deem_10k.idxs', dtype=int)

n_total = 331172
n_train = 10000
n_test = 250
rng = default_rng(seed=11011)
idxs = np.arange(0, n_total)
idxs = np.delete(idxs, idxs_delete)
rng.shuffle(idxs)
train_idxs = idxs[0:n_train]
test_idxs = idxs[n_train:n_train + n_test]

cv_idxs = np.loadtxt('../Processed_Data/DEEM_330k/cv_5.idxs', dtype=int)

In [6]:
# END TMP

In [4]:
# Load SOAP cutoffs
soap_hyperparameters = load_json('../Processed_Data/soap_hyperparameters.json')   
cutoffs = soap_hyperparameters['interaction_cutoff']

In [5]:
# Make directory in which to store models
os.makedirs('../Processed_Data/Models', exist_ok=True)

In [6]:
# Set property names for loading
property_names = ['volumes', 'energies']

# Load structure properties
structure_properties = {}
for pn in property_names:
    structure_properties[pn] = np.loadtxt(f'../Processed_Data/DEEM_330k/Data/structure_{pn}.dat')

# Linear Ridge Regression

In [9]:
# Use all defaults for the template parameters
ridge_parameters = dict()

# Regularization parameters for cross-validation
regularizations = np.logspace(-12, 0, 13)
parameter_grid = dict(ridge__regressor__alpha=regularizations)

In [11]:
# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_330k/Data/{cutoff}'
    
    # Read SOAPs in training set
    sort_idxs = np.argsort(train_idxs) ### TMP
    rev_idxs = np.argsort(sort_idxs) ### TMP
    soaps = utils.load_hdf5(f'{data_dir}/soaps_power_full_avg_nonorm.hdf5', indices=train_idxs[sort_idxs]) ### REMOVE SORT IDXS
    soaps = soaps[rev_idxs] ### TMP
    
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        property_label = pn.capitalize()
        
        # Set working directory
        work_dir = f'../Processed_Data/Models/{cutoff}/Linear_Models/LR/{property_label}'
        os.makedirs(work_dir, exist_ok=True)
        
        # Load the property values (just from the train set)
        y = structure_properties[pn][train_idxs]
         
        # Cross validation pipeline
        cache_dir = mkdtemp(dir=work_dir) # TODO: move this and rmtree outside the property loop?
        pipeline = Pipeline(
            [
                ('norm_scaler', utils.NormScaler()), 
                ('ridge', TransformedTargetRegressor(
                    regressor=Ridge(**ridge_parameters), 
                    transformer=utils.NormScaler()
                ))
            ],
            memory=cache_dir
        )
        gscv = GridSearchCV(
            pipeline, parameter_grid, 
            scoring=[
                'neg_root_mean_squared_error', 
                'neg_mean_absolute_error'
            ],
            cv=utils.cv_generator(cv_idxs),
            refit=False, return_train_score=True, error_score='raise'
        )
        gscv.fit(soaps, y)
        rmtree(cache_dir)
        
        save_json(gscv.cv_results_, f'{work_dir}/cv_results.json', array_convert=True)
        # TODO: check the optimal doesn't give an ill conditioned warning or doesn't converge

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.66057e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.34198e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.40539e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.83784e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.23673e-17): 

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.39509e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.3824e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.40297e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.39643e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.08835e-17): r

In [ ]:
# BEGIN TMP

In [12]:
# Extract optimal parameters as a check

# Loop over cutoffs
for cutoff in cutoffs:
    
    # Loop over properties
    for pn in property_names:
        property_label = pn.capitalize()
        
        work_dir = f'../Processed_Data/Models/{cutoff}/Linear_Models/LR/{property_label}'
        cv_results = load_json(f'{work_dir}/cv_results.json')
        
        # Loop over error types
        for error, error_name in zip(
            ['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
            ['mae', 'rmse']
        ):
        
            idx = np.argmin(cv_results[f'rank_test_{error}'])
            opt_parameters = utils.get_optimal_parameters(cv_results, error, **ridge_parameters)

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error[4:]}-----')
            print(f'{error} =', cv_results[f'mean_test_{error}'][idx])
            print(opt_parameters)
            print('')
            
            # Save optimal parameters for easy access
            save_json(opt_parameters, f'{work_dir}/ridge_parameters_{error_name}.json')

-----Optimal Parameters for 3.5 volumes mean_absolute_error-----
neg_mean_absolute_error = -2.6278993636644223
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -3.763467401447244
{'alpha': 0.001}

-----Optimal Parameters for 3.5 energies mean_absolute_error-----
neg_mean_absolute_error = -0.6811454025198304
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 energies root_mean_squared_error-----
neg_root_mean_squared_error = -0.9701972225669557
{'alpha': 0.0001}

-----Optimal Parameters for 6.0 volumes mean_absolute_error-----
neg_mean_absolute_error = -1.1184560899853364
{'alpha': 0.01}

-----Optimal Parameters for 6.0 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -1.7742455247921032
{'alpha': 0.01}

-----Optimal Parameters for 6.0 energies mean_absolute_error-----
neg_mean_absolute_error = -0.11596155677192237
{'alpha': 1e-07}

-----Optimal Parameters for 6.0 energies root_mean_squared_error-----

In [ ]:
# END TMP

In [14]:
# Extract optimal parameters as a check

# Loop over cutoffs
for cutoff in cutoffs:
    
    # Loop over properties
    for pn in property_names:
        property_label = pn.capitalize()
        
        work_dir = f'../Processed_Data/Models/{cutoff}/Linear_Models/LR/{property_label}'
        cv_results = load_json(f'{work_dir}/cv_results.json')
        
        # Loop over error types
        for error, error_name in zip(
            ['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
            ['mae', 'rmse']
        ):
        
            idx = np.argmin(cv_results[f'rank_test_{error}'])
            opt_parameters = utils.get_optimal_parameters(cv_results, error, **ridge_parameters)

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error[4:]}-----')
            print(f'{error} =', cv_results[f'mean_test_{error}'][idx])
            print(opt_parameters)
            print('')
            
            # Save optimal parameters for easy access
            save_json(opt_parameters, f'{work_dir}/ridge_parameters_{error_name}.json')

-----Optimal Parameters for 3.5 volumes mean_absolute_error-----
neg_mean_absolute_error = -2.549571216699533
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -3.6876161795028453
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 energies mean_absolute_error-----
neg_mean_absolute_error = -0.6690029851874398
{'alpha': 1e-05}

-----Optimal Parameters for 3.5 energies root_mean_squared_error-----
neg_root_mean_squared_error = -0.9511295920625198
{'alpha': 0.001}

-----Optimal Parameters for 6.0 volumes mean_absolute_error-----
neg_mean_absolute_error = -1.0842773569634572
{'alpha': 0.01}

-----Optimal Parameters for 6.0 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -1.7692268855224387
{'alpha': 0.01}

-----Optimal Parameters for 6.0 energies mean_absolute_error-----
neg_mean_absolute_error = -0.1200024862710221
{'alpha': 1e-07}

-----Optimal Parameters for 6.0 energies root_mean_squared_error-----
n

# Kernel Ridge Regression

In [7]:
# Use all defaults for the template parameters
kernel_ridge_parameters = dict(kernel='precomputed')

# Set ranges of kernel gamma (for Gaussian kernel) and regularization
log_gammas = np.linspace(-3, 3, 7)
##log_gammas = np.array([-2.0])

regularizations = np.logspace(-12, 0, 13)
##regularizations = np.array([1.0E-3, 1.0E-2])

# Use the filenames of the kernels that we will be loading
# in the "hacked" pipeline.
# The gamma names for the kernel parsing will be set in the loop
parameter_grid_base = dict(
    ridge__regressor__alpha=regularizations, 
)

In [8]:
# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_10k/Data/{cutoff}'
    model_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian'
    os.makedirs(model_dir, exist_ok=True)
    
    # Read SOAPs in training set
    soaps = utils.load_hdf5(f'{data_dir}/soaps_power_full_nonorm.hdf5') # Use this
    soaps = np.array([np.mean(soap, axis=0) for soap in soaps]) # Use this
    ##str_idxs = [str(i).zfill(4) for i in range(0, len(train_idxs))]
    ##soaps = utils.load_hdf5(f'{data_dir}/soaps.hdf5', datasets=str_idxs, concatenate=False)
    
    # Build a "superkernel": a concatenation of the train and test kernels
    # (which we can store easily in memory and on disk) so that
    # the pipeline doesn't compute whole new kernels for each CV set.
    # We still have to compute a kernel for each gamma, though.
    # We store the kernels named by the logarithm of the gamma parameter,
    # so the filenames don't get ridiculous
    ##XA = np.random.rand(10000, 1000)
    for log_gamma in tqdm(log_gammas, desc='Gamma', leave=False):
        gamma = 10 ** log_gamma
        K = gaussian_kernel(soaps, soaps, gamma=gamma) # Use this
        ##K = gaussian_kernel(XA, XA, gamma=gamma)
        utils.save_hdf5(
            f'{model_dir}/gaussian_kernel_{log_gamma}.hdf5', K, 
            attrs=dict(gamma=gamma, log_gamma=log_gamma), chunks=(100, 100)
        )

In [9]:
# Hacked CV indices so we can load and access the kernels by indexing
# instead of recomputing them
dummy_cv_idxs = np.arange(0, cv_idxs.shape[-1]).reshape((1, -1))

In [11]:
# Optimize hyperparameters

# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    # Set data directory
    model_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian'
    
    parameter_grid = deepcopy(parameter_grid_base)
    parameter_grid['kernel_loader__filename'] = \
        [f'{model_dir}/gaussian_kernel_{log_gamma}.hdf5' for log_gamma in log_gammas]
          
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        property_label = pn.capitalize()
        
        # Set working directory
        work_dir = f'{model_dir}/KRR/{property_label}'
        os.makedirs(work_dir, exist_ok=True)
                
        # Load the property values (just from the train set)
        y = structure_properties[pn][train_idxs]
        
        # "Hacked" pipeline: instead of recomputing the expensive kernels
        # at each CV iteration, we will fit instead with a set of indices
        # and use the filename containing the kernel at a particular
        # gamma as a hyperparameter for KernelLoader, which will
        # load the kernel at initialization.
                
        # NOTE: can't just use the custom kernel as a callable
        # to KernelRidge, as the custom kernel necessarily operates on 2D arrays
        # of the features for all environments in a given structure,
        # whereas the callable must operate on pairs of samples.
        # The KernelLoader/KernelConstructor is used instead
        cache_dir = mkdtemp(dir=work_dir)
        pipeline = Pipeline(
            [
                ('kernel_loader', utils.KernelLoader()),
                ('kernel_norm_scaler', utils.KernelNormScaler()), 
                ('ridge', TransformedTargetRegressor(
                    regressor=KernelRidge(**kernel_ridge_parameters),
                    transformer=utils.SampleSelector(X=y, model=utils.NormScaler()),
                    check_inverse=False
                ))
            ],
            memory=cache_dir
        )
        
        # "Hacked" CV: since we want to access the kernels via indices,
        # we fit the CV with cv_indices, and the cv_generator 
        # is used to pick out the correct folds
        gscv = GridSearchCV(
            pipeline, parameter_grid, 
            scoring=dict(
                neg_mean_absolute_error=make_scorer(
                    utils.score_by_index,
                    greater_is_better=False,
                    y=y, scorer=mean_absolute_error
                ),
                neg_root_mean_squared_error=make_scorer(
                    utils.score_by_index,
                    greater_is_better=False,
                    y=y, scorer=mean_squared_error,
                    squared=False
                )
            ),
            cv=utils.cv_generator(dummy_cv_idxs),
            refit=False, return_train_score=True, error_score='raise'
        )
        gscv.fit(cv_idxs.T, cv_idxs.T)
        save_json(gscv.cv_results_, f'{work_dir}/cv_results.json', array_convert=True)
        rmtree(cache_dir)

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 1.00s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.66s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.


2000 4.960045460337438
2000 6.539265800221978
8000 0.007166189849750411
8000 0.009392341815921012


/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.99s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.64s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.


2000 4.942079096059853
2000 6.43008450991217
8000 0.007181076673697766
8000 0.009410821805014848


/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.99s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.63s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.


...

2000 4.515232739267379
2000 5.424421802057503
8000 0.006555688084134317
8000 0.00783352437125129


/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.97s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.63s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.


2000 4.496065842492748
2000 5.345421404588377
8000 0.006576645948995292
8000 0.007861375018359547


/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.99s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/pipeline.py:313: UserWarning: Persisting input arguments took 0.64s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.


2000 4.486501334937917
2000 5.392068834042245
8000 0.006550265980625226
8000 0.007850006534127258
2000 4.401412512842515
2000 5.262705825404646
8000 0.06497304630108897
8000 0.0778512897570538
2000 4.41522173747049
2000 5.271624863803656
8000 0.06492891516984674
8000 0.07784542723723505
2000 4.514112441602189
2000 5.42264727172415
8000 0.0646010844020409
8000 0.07718730290933713
2000 4.494938557694049
2000 5.343808076466818
8000 0.06480752593875196
8000 0.07746236419478778
2000 4.485312168294686
2000 5.390281419199412
8000 0.0645489751580883
8000 0.07735134825282045



In [55]:
# Extract optimal parameters as a check

# Loop over cutoffs
for cutoff in cutoffs:
    if cutoff != 3.5:
        continue
    
    # Loop over properties
    for pn in property_names:
        property_label = pn.capitalize()
        
        work_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian/KRR/{property_label}'
        cv_results = load_json(f'{work_dir}/cv_results.json')
        
        # Loop over error types
        for error, error_name in zip(
            ['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
            ['mae', 'rmse']
        ):
        
            idx = np.argmin(cv_results[f'rank_test_{error}'])
            opt_parameters = utils.get_optimal_parameters(cv_results, error, **kernel_ridge_parameters)

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error[4:]}-----')
            print(f'{error} =', cv_results[f'mean_test_{error}'][idx])
            print(opt_parameters)
            print('')
            
            # Save optimal parameters for easy access
            save_json(opt_parameters, f'{work_dir}/ridge_parameters_{error_name}.json')

-----Optimal Parameters for 3.5 volumes mean_absolute_error-----
neg_mean_absolute_error = -50.46811285596522
{'kernel': 'precomputed', 'filename': '../Processed_Data/Models/3.5/Kernel_Models/Gaussian/gaussian_kernel_1.0.hdf5', 'alpha': 1e-12}

-----Optimal Parameters for 3.5 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -50.836986007724846
{'kernel': 'precomputed', 'filename': '../Processed_Data/Models/3.5/Kernel_Models/Gaussian/gaussian_kernel_1.0.hdf5', 'alpha': 1e-12}

-----Optimal Parameters for 3.5 energies mean_absolute_error-----
neg_mean_absolute_error = -12395.5157030061
{'kernel': 'precomputed', 'filename': '../Processed_Data/Models/3.5/Kernel_Models/Gaussian/gaussian_kernel_1.0.hdf5', 'alpha': 1e-12}

-----Optimal Parameters for 3.5 energies root_mean_squared_error-----
neg_root_mean_squared_error = -12395.516743897222
{'kernel': 'precomputed', 'filename': '../Processed_Data/Models/3.5/Kernel_Models/Gaussian/gaussian_kernel_1.0.hdf5', 'alpha': 1e-12}

-